# Utilisation de VGG-16 pré-entraîné

Ressource 2.5 -https://openclassrooms.com/fr/courses/4470531-classez-et-segmentez-des-donnees-visuelles/5097666-tp-implementez-votre-premier-reseau-de-neurones-avec-keras

In [1]:
# On importe et charge un modèle pré-entrainé : VGG-16

from keras.applications.vgg16 import VGG16

model = VGG16() # Création du modèle VGG-16 implementé par Keras

553467904/553467096 [==============================] - 58s 0us/step


In [3]:
# Là on vérifie juste que VGG prédit bien. On teste avec une img, mais il faut la préprocesser avant de
# pouvoir la tester dans le modèle

from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

img = load_img('../data/poule1.jpeg', target_size=(224, 224))  # Charge l'image et la redimensionne

img = img_to_array(img)  # Keras traite les images comme des tableaux numpy : convertit l'image chargée
# en tableau numpy

# Le réseau doit recevoir en entrée une collection d'images, stockée dans un tableau de 4 dimensions,
# où les dimensions correspondent dans l'ordre à (nombre d'images, largeur, hauteur, profondeur).
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
# reshape ajoute la première dimension (nombre d'images = 1) à notre image

img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16

In [4]:
y = model.predict(img)  # Prédir la classe de l'image (parmi les 1000 classes d'ImageNet)
# ca renvoit une liste de 1000 proba (une par classe/output)

In [5]:
from keras.applications.vgg16 import decode_predictions

# Afficher les 3 classes les plus probables
print('Top 3 :', decode_predictions(y, top=3)[0])

40960/35363 [==================================] - 0s 0us/step
Top 3 : [('n01514859', 'hen', 0.95390224), ('n01514668', 'cock', 0.044867624), ('n01807496', 'partridge', 0.00091887394)]


In [6]:
# Le modèle est sur à 95% que c'est une poule et 4% que c'est un coq.

# Dataset

Ressource 2.6 : https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

## On charge les données

In [8]:
# On importe la classe qu'on veut. Elle permet de créer pleins de
# variants d'une même image - intéressant quand on a peu d'images
from keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator() # on peut lui proposer pleins
# de paramètres sur les variants qu'on souhaite obtenir

In [11]:
# On charge les données qui sont dans le ML5/data folder grâce à flow_from_directory

train_generator = train_datagen.flow_from_directory(
    directory=r"../data/train/", # chemin du fichier
    target_size=(224, 224), #  size of your input images, every image will be resized to this size
    color_mode="rgb", # if B&W set "grayscale". if 3 color channels, "rgb"
    batch_size=32, # number of images to be yielded from the generator per batch
    class_mode="categorical", # if 2 classes to predict, "binary". if more, "categorical"
    shuffle=True, # shuffle the order of the image being yielded
    seed=42 # random seed
)

Found 6 images belonging to 2 classes.


In [13]:
valid_datagen = ImageDataGenerator()
valid_generator = valid_datagen.flow_from_directory(
    directory=r"../data/valid/", # chemin du fichier
    target_size=(224, 224), #  size of your input images, every image will be resized to this size
    color_mode="rgb", # if B&W set "grayscale". if 3 color channels, "rgb"
    batch_size=32, # number of images to be yielded from the generator per batch
    class_mode="categorical", # if 2 classes to predict, "binary". if more, "categorical"
    shuffle=True, # shuffle the order of the image being yielded
    seed=42 # random seed
)

Found 6 images belonging to 2 classes.


In [14]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    directory=r"../data/test/", # chemin du fichier
    target_size=(224, 224), #  size of your input images, every image will be resized to this size
    color_mode="rgb", # if B&W set "grayscale". if 3 color channels, "rgb"
    batch_size=32, # number of images to be yielded from the generator per batch
    class_mode="categorical", # if 2 classes to predict, "binary". if more, "categorical"
    shuffle=True, # shuffle the order of the image being yielded
    seed=42 # random seed
)

Found 6 images belonging to 1 classes.


## On entraîne et évalue le modèle, puis on prédit

In [15]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

/home/d5learner-05/anaconda3/envs/keras_gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

Ressources supplémentaire : https://www.tensorflow.org/tutorials/images/transfer_learning